In [103]:
import os
import pdfplumber
import glob
import re
import pandas as pd
import numpy as np
import string

In [25]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/Users/quinnunderriner/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


### Extract affordable housing financing data from Ohio PDFs


In [9]:
#filepath to PDFs
filepath = 'Ohio 2019 Applications/'
# create a list of filepaths 
files = glob.glob(filepath+"*.pdf", recursive = True)

In [15]:
test = "AmberlySquareApts.pdf"

In [22]:
pdf = pdfplumber.open(test)
#print(len(pdf.pages[0].extract_tables()[0]))
df = pd.DataFrame(pdf.pages[0].extract_tables()[0])

In [59]:
wage = df[38:][1][51] 


In [63]:
wage[17:]

'Ohio Prevailing Wage'

The pdf reader does not correctly extract tables from these documents, however it does pull data out in a relativley predictable pattern. Below I check for files that have the same structure - only 4 out of 121 do not, so instead of worry about the edge cases I store them in the list "broken files" and will deal with those manually.

In [109]:
for file in files:
    pdf = pdfplumber.open(file)
    #print(len(pdf.pages[0].extract_tables()[0]))
    df = pd.DataFrame(pdf.pages[0].extract_tables()[0])
    if df.shape[0] > 40 and df.iloc[37].values[1] == 'Construction Financing Sources':
        continue
    else:
        print(file, df.shape)

Ohio 2019 Applications/MayflowerManor.pdf (56, 14)
Ohio 2019 Applications/BerwynEastPlace.pdf (25, 9)
Ohio 2019 Applications/AppleseedSeniorApartments.pdf (35, 14)
Ohio 2019 Applications/SW-DetroitShorewayHomes.pdf (35, 13)


In [111]:
broken_files = ["Ohio 2019 Applications/MayflowerManor.pdf",
"Ohio 2019 Applications/BerwynEastPlace.pdf",
"Ohio 2019 Applications/AppleseedSeniorApartments.pdf",
"Ohio 2019 Applications/SW-DetroitShorewayHomes.pdf"]

files = list(set(files) - set(broken_files)) #remove broken files from master set of files 

In [25]:
def scrape_ohio(files):
    """
    Inputs:
        Files (list): List of full filepaths to the PDFs for Virginia. 
    This file notices that while the extract table function of the PDF reader does not exactly work
    in capturing the correct tables, the data follows a similar pattern and so it slices into 
    the files and grabs the relevent info from "Construction Financing Sources" 
    and "Permenant Financing Sources"
    """
    money = []
    wage_requirement = []
    for file in files:
        pdf = pdfplumber.open(file)
        df = pd.DataFrame(pdf.pages[0].extract_tables()[0])
        money_type = df.loc[37,1] #construction
        file_name = re.split("/", file)[-1] #regex to pull the filename from the filepath
        file_name = file_name[0:-4]

        for row in df[38:][1]:
            if 'TOTAL' in row:
                break
            else:
                print(row, "i am a row")
                money.append([x.strip() for x in row.split('$')]+[money_type]+[file_name])
                wage_r = df[38:][1][51][17:]
                wage_requirement.append(wage_r + [file_name])
        money_type = df.loc[37,6] #permanent
        for row in df[38:][6]:
            if 'TOTAL' in row:
                break
            else:
                money.append([x.strip() for x in row.split('$')]+[money_type]+[file_name])

    money_df = pd.DataFrame(money, columns=['Source',"Amount of Funds","Funding Type","File Name"])
    money_df = money_df[~money_df["Amount of Funds"].str.contains("-")] #drop the blanks 
    wage_requirement_df = []
     
    
    return money_df 

In [111]:
df = pd.read_csv("Ohio_scraped_financing_info.csv")

In [136]:
#awards = awards.str.replace(' ', '')
#awards = awards.iloc[1:]

#drop duplicates 
df1 = df[~df["File Name"].str.contains("(1)")]
df = pd.read_csv("Ohio_scraped_financing_info.csv")

In [144]:
#write to csv
df1.to_csv("Ohio_scraped_financing_info.csv")

In [ ]:
df = pd.read_csv("Ohio_scraped_financing_info.csv")
#read in list of those who actually won awards, this will also filter out the 4% and bond gap programs 
awards = pd.read_csv("2019-HTC-CompetitiveAwards.csv")
# link to data: https://ohiohome.org/ppd/documents/2019-HTC-CompetitiveAwards.xlsx
awards = awards["Unnamed: 1"]#drop this if col doesn't exist
awards = awards.dropna()
#now we have list of just those we want to keep 
awards = awards.str.replace(' ', '') #drop whitespace so place names match
awards = awards.iloc[1:]
df = df[df["Source"]!='GP Capital Contribution']
df = df[df["Source"]!='GP Equity'] #drop GP contributions 

#filter for only ones we want 
df = df[df['File Name'].isin(awards)]
df.to_csv("Ohio_scraped_financing_info_Awarded_Only.csv")

In [123]:
df.to_csv("Ohio_scraped_financing_info_Awarded_Only.csv")

In [3]:
df = pd.read_csv("Ohio_scraped_financing_info_Awarded_Only.csv")

In [6]:
#look at the different funding sources to see what needs to be mapped 
df["Source"].value_counts()

Tax Credit Equity                    56
Deferred Developer Fee               44
Other1                               41
Construction Loan                    29
Other2                               29
Permanent First Loan, Hard Debt      24
Other3                               16
HDAP: OHTF/HOME                      11
HDAP                                 11
Other4                                8
Permanent Second Loan                 7
Other5                                5
OHFA HDL                              3
Historic Tax Credit Equity            3
Construction Period Income            2
Cuyahoga County Funds                 2
City of Cleveland Funds               2
Acquired Reserves                     2
Seller Note                           2
GP Equity                             2
City of Columbus/Franklin County      1
FHLB Cin. AHP                         1
DDF and Other                         1
Donations from Huntington and CHN     1
Stark County HOME                     1


In [9]:
df = df[df["Source"]!='GP Capital Contribution']
df = df[df["Source"]!='GP Equity'] #drop GP contributions 

In [25]:
#create subcategories for types of financing, below code applies these and then creates a new column 
tax_credit = ['Tax Credit Equity']
historic_tax_credit = ['Historic Tax Credit Equity']
bank_loan = ['Construction Loan','Permanent First Loan, Hard Debt','Permanent Second Loan','Seller Note'] #for affordable housing trust http://hztrust.org/
affordable_housing_trust = ['Affordable Housing Trust']
deferred_dev = ['Deferred Developer Fee','DDF & Other','DDF and Other']
state_credit = ['HDAP', 'HDAP: OHTF/HOME']  #without specifying which type of HDAP its hard to know if its a grant or loan, but seems liek more grant funding than loans https://ohiohome.org/ppd/documents/HDAP-Matrix-2020.pdf
federal_credit = []
federal_home_loan_bank = ['FHLB AHP','FHLB Cin. AHP']
local_grant = ['City of Cleveland Funds', 'Cuyahoga County Funds''City of Columbus/Franklin County','Donations',
       'City of Columbus', 'Franklin County','Stark County HOME', 'Columbus City HOME','Stark County HOME','Donations from Huntington and CHN','Donations'] #Home are grants https://www.bizjournals.com/columbus/news/2020/09/01/affordable-housing-programs-tax-credits-and-home.html, there was only 1 donation and I put it here but I'm not sure  
public_private_partnership = [] 
fed_grant = []

#unmapped Others, 'Construction Period Income' (shows up twice), 'Acquired Reserves' (also shows up twice)

In [26]:
conditions = [
    df["Source"].isin(tax_credit),
    df["Source"].isin(historic_tax_credit),
    df["Source"].isin(bank_loan),
    df["Source"].isin(affordable_housing_trust),
    df["Source"].isin(deferred_dev),
    df["Source"].isin(state_credit),
    df["Source"].isin(federal_credit),
    df["Source"].isin(federal_home_loan_bank),
    df["Source"].isin(local_grant),
    df["Source"].isin(public_private_partnership),
    df["Source"].isin(fed_grant)]

outputs = ["Tax Credit","Historic Tax Credit",'Bank Loan',"Affordable Housing Trust","Deferred Developer Fee","State Housing Credit Equity",
    "Federal Housing Credit Equity","Federal Home Loan Bank","Local Grant","Public Private Partnership","Federal Grant"]

In [28]:
#this maps the funding sources above
df["Financing_sub_cat"] = np.select(conditions, outputs, 'Other')

In [33]:
#drop unneeded index col, write out to csv 
df.drop(["Unnamed: 0"],axis=1).to_csv("Ohio_scraped_financing_info_Awarded_Only.csv")

In [14]:
awarded = pd.read_csv("Ohio_scraped_financing_info_Awarded_Only.csv")

In [169]:
def extract_from_ohio_extra_info(files):
    """
    Inputs:
        files (list) - strings of path to excel files
    
    This function scrapes info listed under col names from ohio lihtc files  
    """
    cols = ['Project Name', 'Wage Requirement', 'Project Population',"Building Type", "Construction Type","Census Tract", 
            "Total Project Cost","Total Units"]
    final_df = pd.DataFrame(columns=cols)
    df = []
    for file in files:
        #print("starting file: ", file)
        file_name = re.split("/", file)[-1]
        file_name = file_name[0:-4]
        
        try:
            pdf = pdfplumber.open(file)
            #print(len(pdf.pages[0].extract_tables()[0]))
            project_information = pd.DataFrame(pdf.pages[0].extract_tables()[0])
        except Exception as e:
            print("cant read data",file_name, e)

            
        try:
            
            new_row = [file_name, project_information[1][51],project_information[1][8].splitlines()[0],
                       project_information[1][8].splitlines()[1],
                      project_information[1][8].splitlines()[2],
                       project_information[1][8].splitlines()[6],project_information[1][48],
                       project_information[6][5].split('units')[0][-10:]+project_information[6][5].split('units')[0][10:]]
            
        except Exception as e:
            print("cant grab data",file_name, e)
   
        try:
            zipped = zip(cols, new_row)
            add_from_dict = dict(zipped)
            df.append(add_from_dict)
        
        except Exception as e:
            print("cant add new info",file_name, e)
        
    final_df = final_df.append(df, True)   
    return final_df

In [170]:
test_take = extract_from_ohio_extra_info(files)
#Only northland gate is a 9% project, can ignore the errors with rest of the readings 

cant grab data ArtsApartmentsatMusicHall 'NoneType' object has no attribute 'split'
cant grab data MayflowerManor list index out of range
cant grab data EastVillage 'NoneType' object has no attribute 'split'
cant grab data PincrestApartments 'NoneType' object has no attribute 'split'
cant grab data BerwynEastPlace 51
cant grab data FieldsErtelTownhomes 'NoneType' object has no attribute 'split'
cant grab data AppleseedSeniorApartments 51
cant grab data BrynMawrGlen 'NoneType' object has no attribute 'split'
cant grab data SW-DetroitShorewayHomes 51
cant grab data NorthlandGate 51
cant grab data AtchesonPlaceLofts (1) 'NoneType' object has no attribute 'split'


In [182]:
len(nine_percent_projects)

30

In [183]:
df_filtered = test_take[test_take["Project Name"].isin(nine_percent_projects)]

In [185]:
# we have data on 30 projects, like we expected to 
len(df_filtered)

30

In [157]:
#need to filter new extracted info for just 9 percent projects 
valid = pd.read_csv("Ohio_Financing_PDF_Extraction/Ohio_scraped_financing_info_Awarded_Only.csv")
#only northland gate was unable to be grabbed and is also a valid 
nine_percent_projects = sorted(list(valid["File Name"].unique()))

df_filtered = test_take[test_take["Project Name"].isin(nine_percent_projects)]

In [249]:
#want to do some final cleaning 
def clean_up_ohio_auxilary_data(df):
    """
    strip out extra text from pdf extraction 
    
    """
    df["Total Project Cost"] = df["Total Project Cost"].str.translate(str.maketrans('', '', string.punctuation)).str.extract('(\d+)')
    df["Total Project Cost"] = df["Total Project Cost"].astype(str).str[:-2].astype(int) #shave off cents part 
    df["Census Tract"] = df["Census Tract"].str.extract('(\d+)')
    df["Building Type"] = df["Building Type"].str[-11:]
    df["Project Population"] = df["Project Population"].str[11:]
    df["Wage Requirement"] = df["Wage Requirement"].str[17:]
    df["Construction Type"] = df["Construction Type"].str[18:]
    df = df.reset_index().drop(columns=["index"])
    
    return df 
    

In [ ]:
import warnings
warnings.filterwarnings('ignore')
df_filtered = clean_up_ohio_auxilary_data(df_filtered)

In [233]:
#Note: bmanually added data from Northland gate, manually pulled out number of units from units text 
df_filtered.to_csv("clean_me.csv")
df_filtered_post_manual = pd.read_csv("ohio_cleaned.csv")

In [255]:
#read in nine percent financing data 
nine_percent = pd.read_csv("Ohio_scraped_financing_info_Awarded_Only.csv")
nine_percent = nine_percent.drop(columns = ["Unnamed: 0"])
#rename to make merge cleaner 
nine_percent = nine_percent.rename(columns={"File Name":"Project Name"})

In [261]:
df_filtered_post_manual.head()

,Project Name,Wage Requirement,Project Population,Building Type,Construction Type,Census Tract,Total Project Cost,Total Units
0,SheltonGardens,Davis Bacon,Families,Multifamily,Construction Type Rehabilitation,39061007700,12984615,137
1,JenkinsStreetLofts,None,Families,Multifamily,Construction Type New Construction,39049005820,11998783,60
2,EasternWoodsSenior,None,Seniors,Multifamily,Construction Type New Construction,39063000200,8879220,50
3,CarpenterHall,0,Families,Multifamily,Construction Type Rehabilitation,39009973902,7700005,53
4,CarpenterHall,0,Families,Multifamily,Construction Type Rehabilitation,39009973902,7700005,53


In [265]:
final_df = df_filtered_post_manual.merge(nine_percent, on="Project Name")

In [271]:
final_df.to_csv("Ohio_Lihtc_9%_financing_final.csv")